In [12]:
#Basic Dependencies
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:

hack=pd.read_csv(r"E:\Data science projects\Machine hack\Great India hackathon\Participants_Data_TGIH\Participants_Data_TGIH\train.csv")

In [6]:
test=pd.read_csv(r"E:\Data science projects\Machine hack\Great India hackathon\Participants_Data_TGIH\Participants_Data_TGIH\test.csv")

In [7]:
hack.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [9]:
hack.drop(["InvoiceNo", "InvoiceDate"],axis=1,inplace=True)

In [10]:
hack.head()

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
0,1583,144,3,3.75,14056.0,35
1,1300,3682,6,1.95,13098.0,35
2,2178,1939,4,5.95,15044.0,35
3,2115,2983,1,0.83,15525.0,35
4,1210,2886,12,1.65,13952.0,35


In [11]:
hack.skew()

StockCode        0.271247
Description     -0.133967
Quantity         0.226115
UnitPrice      438.340508
CustomerID       0.029524
Country         -3.050125
dtype: float64

In [13]:
corr_matrix = hack.corr(method='pearson')
corr_matrix

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
StockCode,1.000000,-0.013230,-0.000942,0.020073,0.002887,0.006242
Description,-0.013230,1.000000,0.000194,-0.000424,-0.005227,-0.014791
Quantity,-0.000942,0.000194,1.000000,-0.000924,-0.002930,-0.008001
UnitPrice,0.020073,-0.000424,-0.000924,1.000000,-0.004932,-0.005378
CustomerID,0.002887,-0.005227,-0.002930,-0.004932,1.000000,0.388198
Country,0.006242,-0.014791,-0.008001,-0.005378,0.388198,1.000000


In [14]:
#Take targate variable into y
y = hack['UnitPrice']
X = hack.drop('UnitPrice',axis = 1)

from sklearn.preprocessing import PowerTransformer
y = y.values.reshape(-1,1)
# power transform the raw data
power = PowerTransformer(method='yeo-johnson', standardize=True)
y = power.fit_transform(y)

In [15]:
# Split data into train and test format
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [17]:
from sklearn.pipeline import Pipeline
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', MinMaxScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', MinMaxScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', MinMaxScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', MinMaxScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', MinMaxScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', MinMaxScaler()),('GBM', GradientBoostingRegressor())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=21)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

ScaledLR: -0.988056 (0.007975)
ScaledLASSO: -0.999830 (0.007032)
ScaledEN: -0.999830 (0.007032)
ScaledKNN: -0.330217 (0.003717)
ScaledCART: -0.072322 (0.002387)
ScaledGBM: -0.518521 (0.005457)


In [18]:
# Using K_Nearest neighbour regressor
# running for different K values to know which yields the max accuracy.
from sklearn.neighbors import KNeighborsRegressor
score = []
for k in range(1,20):    
    clf = KNeighborsRegressor(n_neighbors = k,  weights = 'distance', p=1)
    clf.fit(X_train, y_train)
    score.append(clf.score(X_test, y_test))

In [19]:
k_max = score.index(max(score))+1
print( "At K = {}, Max Accuracy = {}".format(k_max, max(score)*100))

At K = 3, Max Accuracy = 82.8484485052212


In [20]:
clf = KNeighborsRegressor(n_neighbors = k_max,  weights = 'distance', p=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test ))   
y_pred = clf.predict(X_test)

0.828484485052212


In [21]:
import joblib
# save the model to disk
filename = 'KNN_Regressor_MachineHack.sav'
joblib.dump(model, filename)

['KNN_Regressor_MachineHack.sav']

In [22]:
data_test = test.drop(['InvoiceDate','InvoiceNo'],1, inplace= False)
data_test.head()

,StockCode,Description,Quantity,CustomerID,Country
0,1709,1079,3,16161.0,35
1,510,3457,1,17341.0,35
2,604,694,36,15158.0,35
3,1478,3473,2,16033.0,35
4,3216,871,1,15351.0,35


In [23]:
# apply the whole pipeline to data
results = clf.predict(data_test)

In [24]:
result = pd.DataFrame(data=results)
result.head()

,0
0,-0.188574
1,-0.519521
2,0.827562
3,-0.240852
4,1.960363


In [25]:
# Applying inverse power transform to get back original form of unit price
a_inverse_transformed = power.inverse_transform(result)
a_inverse_transformed

array([[1.65      ],
       [1.25      ],
       [3.88417208],
       ...,
       [1.25      ],
       [5.15079852],
       [4.15      ]])

In [33]:
result = pd.DataFrame(data=(a_inverse_transformed).round(2),columns=["UnitPrice"])
result.head()

,UnitPrice
0,1.65
1,1.25
2,3.88
3,1.58
4,12.75


In [34]:
result.columns

Index(['UnitPrice'], dtype='object')

In [30]:
result.to_excel("2.MachineHack_UnitPricePrediction.xlsx")